# AI Exam

Consider the following environment:

<img src="images/road_env.jpg" style="zoom: 40%;"/>

The agent starts in cell $S=(0, 0)$ and must reach the goal in cell $G=(8,6)$. The agent can move in the four directions (except when a wall is present), and each step has unitary cost. 
In cells representing roads with intersections, the agent must wait for the traffic light to turn green before proceeding. At busy intersections (indicated by two traffic lights in the same cell), the agent will have to wait a long time to cross the intersection, greatly increasing the cost of the route (+5). On the other hand, intersections with only one traffic light are less busy and the agent will only pay a cost of 2 to cross them.

Assuming that actions are fully deterministic, consider the problem of finding a minimum cost path from $S$ to $G$. Find a solution using the approach that you consider most suited for the problem and motivate your choice.

In [2]:
import os, sys 

module_path = os.path.abspath(os.path.join('tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gym, envs
from utils.ai_lab_functions import *
import numpy as np
from timeit import default_timer as timer
from tqdm import tqdm as tqdm

env_name = 'RoadEnv-v0'
env = gym.make(env_name)

env.render()

print("\nActions encoding: ", env.actions)

# Remember that you can know the type of a cell whenever you need by accessing the grid element of the environment:
print("Cell type of start state: ",env.grid[env.startstate])
print("Cell type of goal state: ",env.grid[env.goalstate])
state = 15 # a very busy intersection
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])
state = 10 # a less busy intersection
print(f"Cell type of cell {env.state_to_pos(state)}: ",env.grid[state])

[['S' 'R' 'W' 'W' 'W' 'W' 'R' 'W' 'W']
 ['W' 'Ts' 'R' 'R' 'R' 'R' 'Tl' 'R' 'R']
 ['W' 'R' 'W' 'W' 'W' 'W' 'R' 'W' 'W']
 ['R' 'Ts' 'R' 'Ts' 'R' 'R' 'Ts' 'W' 'W']
 ['W' 'W' 'W' 'R' 'W' 'W' 'R' 'Ts' 'R']
 ['W' 'R' 'R' 'Tl' 'W' 'W' 'W' 'R' 'W']
 ['W' 'R' 'W' 'R' 'Ts' 'R' 'R' 'Tl' 'R']
 ['W' 'R' 'W' 'W' 'R' 'W' 'W' 'R' 'W']
 ['R' 'Ts' 'R' 'R' 'Tl' 'R' 'G' 'Ts' 'R']]

Actions encoding:  {0: 'L', 1: 'R', 2: 'U', 3: 'D'}
Cell type of start state:  S
Cell type of goal state:  G
Cell type of cell (0, 0):  S
Cell type of cell (1, 1):  Ts


Remember that it is on you to keep into account the cost of each step!

Here's an example of how you can handle the problem:

In [5]:
# This check can be used to determine the cost of an action:
state = 1 #state in position (0,1)
node=Node(state)
cost = 1 #default value for action cost
for action in range(env.action_space.n):  #Look around
    child_state = env.sample(node.state, action)
    if env.grid[child_state] == 'Tl':
        cost = 5 #if next state is a cell with two traffic lights the cost is 5
    if env.grid[child_state] == 'Ts':
        cost = 2 #if next state is a cell with just one traffic light the cost is 2
    child = Node( 
        child_state, # node state
        node, # parent node
        node.pathcost + cost, # incremental path cost
    ) 
    print(f"State {state}, action {action}, next state {child.state}, child cost {child.pathcost}")

State 1, action 0, next state 0, child cost 1
State 1, action 1, next state 1, child cost 1
State 1, action 2, next state 1, child cost 1
State 1, action 3, next state 10, child cost 2


Use the following code cells to implement and test your solution.

In [12]:
#IMPLEMENT THIS FUNCTION, YOU CAN CHANGE THE PARAMETERS FOR THE FUNCTION IF THIS IS USEFUL
def my_solution(environment, node= None):

    if node == None:
        node= Node(environment.startstate, None, 0)
        
    if environment.goalstate == node.state: # Goal state check
        return build_path(node), 1, node.pathcost

    time_cost= 0
    memory_cost= 0

    blockedOccurred= False
    for action in range(environment.action_space.n):
        next_state= environment.sample(node.state, action)

        parent= node.parent


        if environment.grid[next_state] == 'W' or environment.grid[parent.state]:
            return "blocked", 1, node.pathcost

        if environment.grid[next_state] == 'Tl':
            time_cost= 5 #if next state is a cell with two traffic lights the cost is 5
        if environment.grid[next] == 'Ts':
            time_cost= 2 #if next state is a cell with just one traffic light the cost is 2
        else:
            time_cost+=1
        
        memory_cost+=1
            
        next_node = Node( 
            next_state, # node state
            node, # parent node
            node.pathcost + cost, # incremental path cost
        )             
        
        result= my_solution(environment, next_node)

        time_cost+= result[1]

        if result[0] == "blocked":
            blockedOccurred = True
        elif result[0] != "failure": # Solution found
            return build_path(Node), time_cost, memory_cost
        
    if(cut_off_occurred): # Solution not found but cutoff occurred
        return "blocked", time_cost, memory_cost
    else: # No solution and no cutoff: failure
        return "failure", time_cost, memory_cost

    


In [13]:
env_name = 'RoadEnv-v0'
env = gym.make(env_name)

t = timer()

path, time_cost, memory_cost = my_solution(env)

print("\nEXECUTION TIME: \n{}\n".format(round(timer() - t, 4)))

print("Solution: {}".format(solution_2_string(path, env)))
print("N° of nodes explored: {}".format(time_cost))
print("Max n° of nodes in memory: {}\n".format(memory_cost))

check_sol(solution_2_string(path, env))

AttributeError: 'NoneType' object has no attribute 'state'